Interactome expression landscape for all each cell-type
===========
***
Here we are continuing the sanity test of interaction we found in both Promoter distal region and Promoter-promoter interaction we found. The main aim of this analysis:

* Here we provide substantial evidence of expressed genes in interacting genes makes it more sense. 
* 2 figures each of qqplot of interacting genes and barplot of expressed genes as the function of all the expressed genes.
* Make a Gene ontology enrichment analysis of these expressed interacting genes 
***

### Background to analysis

The approach is based chromatin interaction through genome looping induces the expression of relevant genes for biological processes. This has to somehow come up in our interaction genes that were found with the HiCap analysis. However, it has to benoted that we will not get all the expressed genes with genome looping. There will be certain genes that are trans regulated. 

### Data 
There are in general three types of data that we are looking into:
* Expression data of all genes in FPKM in two replicates
* P-D  interaction dataset
* P-P interaction datast
 
 SO what to do og G-P and G-E dataset? At the moment lets filter these dataset and procced with only those clean version of dataset that can be used. The motive of filtering these dataset is to make the analysis more interpretable. For example if there is genes involved in formation of genomic loop then they should have high expression profile than background set of genes and possibly later we can extrapolate these G-P interaction based on these analysis.
 
***

### Analysis 

Earlier I made a script based on this kind of analysis. Lets see I can make that script work or not
```shell
 awk -F "[\t=;]" '{print $10}' Design6.hg19.AllProbes.MboI.13.23.06_2017-08-17.gff3 |awk '{ if (NR >2) print $0}' | grep -v "rs"  | sort | uniq
```


for the thp1 data remove the unwanted LPS condition
"
cut -f -18 THP1_nLPS_P_D_final.txt > THP1_nLPS_P_D_filfinal.txt 
"
***

```shell

awk '{if ($2 > 0 || $3 > 0) print $0}' gene_expression_THP1.txt > genes_expressed_THP1.txt
awk '{if ($2 == 0 && $3 == 0) print $0}' gene_expression_THP1.txt > genes_unexpressed_THP1.txt 

awk -F "\t" '{if ($3 > 0 || $4 > 0) print $0}' gene_expression_SMC.txt > genes_expressed_SMC.txt
awk -F "\t" '{if ($3 == 0 && $4 == 0) print $0}' gene_expression_SMC.txt > genes_unexpressed_SMC.txt 

````
Total supporting pairs in each of the cell-type

50681600 62030568 
67361183 59558262
88014274 101405122

#### Update on May 4 meeting

Make a PCA plot of both PP and PD interaction revealing difference between expression interaction profile of these interaction with respect to expressed genes


A new uitility script was written that takes all the interaction and expressesed genes and returns the dictionary of these files. this script is named as hicap_exps.py 

Please note that name for the gff files which contain the design information have been changed .


In [1]:
def exp_dict(file):
    """
        The function take expression file and returns dictionary dataforma of the file
        Since the expression values are given here in the last two columns, I have taken 
        
        file = expression.txt 
    
    """
    dict_exp = {}
    with open (file, 'r') as exp_data:
        next(exp_data)
        
        for lines in exp_data:
            line = lines.strip().split("\t")
            dict_fields = line[0]
            fields = dict_exp.get(dict_fields,"")
            fields = line[-2],line[-1]
            dict_exp[dict_fields] = fields
    return(dict_exp)


def int_dict_pd(file):
    """
        This function takes the interaction file of promoter distal interaction and returns dictionary of gene name as 
        the key and transcript_Id, distal chromosome position, as the list.
        
        file = interaction file from HiCap runs with RefSeqName as the gene_ID, TranscriptName as transcript_ID
    
    """
    results_interaction = {}
    with open (file, 'r') as int_files:
        next(int_files)
        
        for lines in int_files:
            line = lines.strip()
            fields = line.split("\t")
            if fields[11] == "-1":
                pass
            else:
                interaction_genes = fields[0]
                interaction_status_genes = results_interaction.get(interaction_genes,[])
                interaction_status_genes.append([fields[1], fields[8], fields[9], fields[10], fields[11]])
                results_interaction[interaction_genes] = interaction_status_genes
    return (results_interaction)


def int_dict_pp(file):
    """
        This function takes the interaction file of promoter promoter interaction and returns dictionary of gene name as 
        the key and transcript_Id, distal chromosome position, as the list.
        
        file = interaction file from HiCap runs with RefSeqName as the gene_ID, TranscriptName as transcript_ID
    
    """
    results_interact_PP = {}
    with open (file, 'r') as int_files:
        next(int_files)
        
        for lines in int_files:
            line = lines.strip()
            fields = line.split("\t")
            interaction_genes = fields[0]
            interaction_status_genes = results_interact_PP.get(interaction_genes,[])
            interaction_status_genes.append([fields[1], fields[11], fields[12], fields[13], fields[16]])
            results_interact_PP[interaction_genes] = interaction_status_genes
    return (results_interact_PP)

def file2array(file):
    array =[]
    with open (file,'r') as files:
        for lines in files:
            line = lines.strip()
            array.append(line)
    return(array)


In [2]:
def enhancer_RPKP(file):
    """
        This function takes the interaction file of promoter distal interaction and returns dictionary of gene name as 
        the key and transcript_Id, distal chromosome position, as the list.
        
        file = interaction file from HiCap runs with RefSeqName as the gene_ID, TranscriptName as transcript_ID
    
    """
    results_interaction = {}
    counts_rep1 =0
    counts_rep2 =0
    final_list ={}
    with open (file, 'r') as int_files:
        next(int_files)
        
        for lines in int_files:
            line = lines.strip()
            fields = line.split("\t")
            if fields[11] == "-1":
                pass
            else:
                interaction_genes = fields[0]
                length = abs(int(fields[9])- int(fields[10]))
                interaction_status_genes = results_interaction.get(interaction_genes,[])
                interaction_status_genes.append( [fields[12], fields[15], length])
                results_interaction[interaction_genes] = interaction_status_genes

    for k in results_interaction.values():
        for value in k:
            counts_rep1 = int(counts_rep1) + int(value[0])
            counts_rep2 = int(counts_rep2) + int(value[1])
    
    for k in results_interaction.keys():
        Enh_rep1 = 0
        Enh_rep2 = 0
        tot_len_Enh = 0
        for values in results_interaction[k]:
            Enh_rep1 = int(values[0]) + Enh_rep1
            Enh_rep2 = int (values[1]) + Enh_rep2
            tot_len_Enh = int(values[2])+ tot_len_Enh
        tot_len_Enh = tot_len_Enh/1000
        Enh_rep1 = round(Enh_rep1/(counts_rep1/1000000),3)
        Enh_rep2 = round(Enh_rep2/(counts_rep2/1000000),3)
        #Enh_rep1 = round(Enh_rep1/((counts_rep1/1000000)*tot_len_Enh),3)
        #Enh_rep2 = round(Enh_rep2/((counts_rep2/1000000)*tot_len_Enh),3)
        final_list[k] = [Enh_rep1,Enh_rep2,tot_len_Enh]
    return (final_list, counts_rep1, counts_rep2)

***

In [ ]:
"""
import os 
os.chdir ("//Volumes/Work_drive/prj/Expression_Vs_Interaction/data/BAV/")

expressed_genes_BAV = exp_dict("gene_expression_filter_header.txt")
interaction_BAV_pd = int_dict_pd ("BAVrun.hg19.Proximities.Probe_Distal.filtered.txt")
enhancer_BAV_RPKM = enhancer_RPKP("BAVrun.hg19.Proximities.Probe_Distal.filtered.txt")
"""

In [4]:

import os 
os.chdir ("/Volumes/Work_drive/prj/Expression_Vs_Interaction/data/THP1/")

expressed_genes_THP1 = exp_dict("gene_expression_THP1_nlps_filtered.txt")
Genes_probes = file2array("Genes_probes_THP1.txt")
#interaction_THP1_pd = int_dict_pd ("THP1run.hg19.nLPS.Proximities.Probe_Distal.filtered.txt")
interaction_THP1_pd = int_dict_pd ("/Volumes/Work_drive/prj/final_interaction_table/THP1_nLPS_P_D_filfinal.txt")
interaction_THP1_pp = int_dict_pp("THP1run.hg19.nLPS.Proximities.Probe_Probe.SMCscaled.filtered.txt")
#enhancer_THP1_RPKM = enhancer_RPKP("THP1run.hg19.nLPS.Proximities.Probe_Distal.filtered.txt")
enhancer_THP1_RPKM, TCRep1,TCRep2 = enhancer_RPKP("/Volumes/Work_drive/prj/final_interaction_table/THP1_nLPS_P_D_filfinal.txt")


In [5]:
print(TCRep1,TCRep2)

497722 458880


In [3]:
"""
import os 
os.chdir ("/Volumes/Work_drive/prj/Expression_Vs_Interaction/data/TAV/")

expressed_genes_TAV = exp_dict("gene_expression_filtered_TAV.txt")
Genes_probes = file2array("Genes_probes_TAV.txt")
#interaction_TAV_pd = int_dict_pd ("TAVrun.hg19.Proximities.Probe_Distal.filtered.txt")
interaction_TAV_pd = int_dict_pd ("/Volumes/Work_drive/prj/final_interaction_table/TAV_hg19_P_D_final.txt")
interaction_TAV_pd = int_dict_pd ("TAVrun.hg19.Proximities.Probe_Distal.filtered.txt")
interaction_TAV_pp = int_dict_pp("TAVrun.hg19.Proximities.Probe_Probe.SMCscaled.filtered.txt")
#enhancer_TAV_RPKM = enhancer_RPKP("TAVrun.hg19.Proximities.Probe_Distal.filtered.txt")
enhancer_TAV_RPKM, TCRep1,TCRep2 = enhancer_RPKP("/Volumes/Work_drive/prj/final_interaction_table/TAV_hg19_P_D_final.txt")

"""

In [4]:
print(TCRep1,TCRep2)

1144866 911035


In [15]:
"""
import os 
os.chdir ("/Volumes/Work_drive/prj/Expression_Vs_Interaction/data/SMC/")

expressed_genes_VSMC = exp_dict("gene_expression_VSMC_filtered.txt")
Genes_probes = file2array("Genes_probes_SMC.txt")
#interaction_VSMC_pd = int_dict_pd ("HASMC.hg19.Proximities.Probe_Distal.filtered.txt")
interaction_VSMC_pd = int_dict_pd ("/Volumes/Work_drive/prj/final_interaction_table/HASMC_hg19_P_D_final.txt")
interaction_VSMC_pp = int_dict_pp("SMC.hg19.Proximities.Probe_Probe.filtered.txt")
#enhancer_VSMC_RPKM = enhancer_RPKP("HASMC.hg19.Proximities.Probe_Distal.filtered.txt")
enhancer_VSMC_RPKM, TCRep1,TCRep2 = enhancer_RPKP("/Volumes/Work_drive/prj/final_interaction_table/HASMC_hg19_P_D_final.txt")
"""

In [16]:
print(TCRep1,TCRep2)

483416 725378


***

In [6]:
fh = open("Int_Exp_Genes_probes.txt", "w") 
for keys, values in expressed_genes_VSMC.items():
    if keys in Genes_probes:
        fh.write(keys+"\t"+"\t".join(values))
        fh.write ("\n")
fh.close()                 

In [7]:
fh = open("No_Int_Exp_Genes_probes.txt", "w") 
for keys, values in expressed_genes_VSMC.items():
    if keys not in Genes_probes:
        #print (keys+"\t"+"\t".join(values))
        fh.write(keys+"\t"+"\t".join(values))
        fh.write ("\n")
fh.close()  

In [6]:
"""
    This is just a command to get the interaction status in each gene. Just run once in order to get the interaction 
    list.

import sys
sys.stdout = open('Gene_int_THP1_count_PD.txt', 'w')

for keys, values in interaction_THP1_pd.items():
    print (keys+"\t"+str(len(values)))
    
import sys
sys.stdout = open('Gene_int_VSMC_count_PD.txt', 'w')

for keys, values in interaction_VSMC_pd.items():
    print (keys+"\t"+str(len(values)))
    
import sys
sys.stdout = open('Gene_int_TAV_count_PD.txt', 'w')

for keys, values in interaction_TAV_pd.items():
    print (keys+"\t"+str(len(values)))


import sys
sys.stdout = open('Gene_int_BAV_count_PD.txt', 'w')

for keys, values in interaction_BAV_pd.items():
    print (keys+"\t"+str(len(values)))
    
"""

fh = open("Gene_int_THP1_count_PD_2.txt", "w") 
for keys, values in interaction_THP1_pd.items():
    if keys in enhancer_THP1_RPKM.keys():
        fh.write(keys+"\t"+str(len(values))+"\t"+"\t".join([str(i) for i in  enhancer_THP1_RPKM[keys]]))
    fh.write ("\n")
fh.close()  


In [ ]:
74.538/4227.46

***

In [ ]:
"""
    Thia is just a command to get the interaction status in each gene. Just run once in order to get the interaction 
    list.

import sys
sys.stdout = open('Gene_int_THP1_count_PP.txt', 'w')

for keys, values in interaction_THP1_pp.items():
    print (keys+"\t"+str(len(values)))


import sys
sys.stdout = open('Gene_int_VSMC_count_PP.txt', 'w')

for keys, values in interaction_VSMC_pp.items():
    print (keys+"\t"+str(len(values)))
"""


fh = open("Gene_int_VSMC_count_PP.txt", "w") 
for keys, values in interaction_VSMC_pp.items():
    fh.write(keys+"\t"+str(len(values)))
    fh.write ("\n")
fh.close()  

In [9]:
"""
    This is just a command to get the interaction status in each gene. Just run once in order to get the interaction 
    list.
"""

fh = open("VSMC_Probe_Distal.filtered_expression.txt", "w")

with open ("/Volumes/Work_drive/prj/final_interaction_table/HASMC_hg19_P_D_final.txt", 'r') as Int:
    for lines in Int:
        
        if lines.startswith('RefSeqName'):
            line = lines.strip().split("\t")
            line.append ('Exp_rep1_FPKMS')
            line.append ('Exp_rep2_FPKMS')
            line.append ('Int_rep1_RPKM_interation')
            line.append ('Int_rep1_RPKM_interation')
            line.append ('Total_enhancers_length')
            myorder = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22] 
            line = [str(line[i]) for i in myorder]
            fh.write("\t".join(line))
            fh.write ("\n")
            continue
            
        else:
            line = lines.strip().split("\t")
            
            if line[11] == "-1":
                pass
            else:
                if line[0] in expressed_genes_VSMC.keys() and line [0] in enhancer_VSMC_RPKM.keys():
                    exp_fields = expressed_genes_VSMC[line[0]]
                    FPKM_interaction = enhancer_VSMC_RPKM[line[0]]
                    for i in exp_fields:
                        line.append(i)
                    for j in FPKM_interaction:
                        line.append(j)
                    myorder = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]  
                    line = [str(line[i]) for i in myorder]
                    fh.write("\t".join(line))
                    fh.write ("\n")

                else:
                #FPKM_interaction = enhancer_THP1_RPKM[line[0]]
                    line.append ('-1')
                    line.append ('-1')
                    for j in FPKM_interaction:
                        line.append(j)
                    myorder = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22] 
                    line = [str(line[i]) for i in myorder]
                    fh.write("\t".join(line))
                    fh.write ("\n")
fh.close()                 

***

In [ ]:
"""
    This is just a command to get the interaction status in each gene. Just run once in order to get the interaction 
    list. Not run yet ...
"""
sys.stdout = open('TAV_Probe_Probe.filtered_expression.txt', 'w')

with open ("TAVrun.hg19.Proximities.Probe_Probe.SMCscaled.filtered.txt", 'r') as Int:
    for lines in Int:
        
        if lines.startswith('RefSeqName'):
            line = lines.strip().split("\t")
            line.append ('TAV_rep1_FPKMS')
            line.append ('TAV_rep2_FPKMS')
            myorder = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24] 
            line = [line[i] for i in myorder]
            print ("\t".join(line))
            continue
            
        else:
            line = lines.strip().split("\t")
            if line[0] in expressed_genes_VSMC.keys():
                exp_fields = expressed_genes_VSMC[line[0]]
                for i in exp_fields:
                    line.append(i)
                myorder = myorder = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24] 
                line = [line[i] for i in myorder]
                print ("\t".join(line))
            else:
                line.append ('-1')
                line.append ('-1')
                line.append ('-1')
                myorder = myorder = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]  
                line = [line[i] for i in myorder]
                print ("\t".join(line))

In [ ]:
%%bash

(head -n 1 TAV_Probe_Distal.filtered_expression.txt && tail -n +2 TAV_Probe_Distal.filtered_expression.txt |sort -k 1n) > sorted_TAV_Probe_Distal_filtered_expression.txt


In [ ]:
%%bash

(head -n 1 TAV_Probe_Probe.filtered_expression.txt && tail -n +2 TAV_Probe_Probe.filtered_expression.txt |sort -k 1n) > sorted_TAV_Probe_probe_filtered_expression.txt

In [ ]:
#####################################################################################################################################################
setwd("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/")
Genes_target_THP1 <- read.table("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/THP1_PD_SP", quote="\"", comment.char="")
Genes_target_THP1$V1 <- as.character(Genes_target_THP1$V1)
Genes_target_TAV <- read.table("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/TAV_PD_SP", quote="\"", comment.char="")
Genes_target_TAV$V1 <- as.character(Genes_target_TAV$V1)
Genes_target_SMC <- read.table("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/SMC_PD_SP", quote="\"", comment.char="")
Genes_target_SMC$V1 <- as.character(Genes_target_SMC$V1)

Comm_probe_genes = Reduce(intersect,  list(Genes_target_THP1$V1, 
                                           Genes_target_TAV$V1,
                                           Genes_target_SMC$V1))

write.table(unique(Comm_probe_genes),"Common_ProbeGenes_targetted.txt",sep="\t", quote=FALSE,
            row.names=FALSE, col.names = FALSE)

#####################################################################################################################################################
setwd("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/expression/")
genes_expressed_THP1 <- read.delim("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/expression/genes_expressed_THP1.txt")
colnames (genes_expressed_THP1) <- c("gene","FPKM_THP1_Rep_1","FPKM_THP1_Rep_2")
genes_expressed_THP1$gene <- as.character(genes_expressed_THP1$gene)


genes_expressed_TAV <- read.delim("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/expression/genes_expressed_TAV.txt")
colnames (genes_expressed_TAV) <- c("gene","transcipt_ID","FPKM_TAV_Rep_1","FPKM_TAV_Rep_2")
genes_expressed_TAV$gene <- as.character(genes_expressed_TAV$gene)


genes_expressed_SMC <- read.delim("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/expression/genes_expressed_SMC.txt")
colnames (genes_expressed_SMC) <- c("gene","transcript_ID","FPKM_SMC_Rep_1","FPKM_SMC_Rep_2")
genes_expressed_SMC$gene <- as.character(genes_expressed_SMC$gene)


Comm_expressed_genes = Reduce(intersect,  list(genes_expressed_THP1$gene, 
                                               genes_expressed_TAV$gene,
                                               genes_expressed_SMC$gene))

write.table(unique(Comm_expressed_genes),"Common_Expressed_gene.txt",sep="\t", quote=FALSE,
            row.names=FALSE, col.names = FALSE)

#####################################################################################################################################################
setwd("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/expression/")
genes_Unexpressed_THP1 <- read.delim("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/expression/genes_unexpressed_THP1.txt")
colnames (genes_Unexpressed_THP1) <- c("gene","FPKM_THP1_Rep_1","FPKM_THP1_Rep_2")
genes_Unexpressed_THP1$gene <- as.character(genes_Unexpressed_THP1$gene)


genes_Unexpressed_TAV <- read.delim("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/expression/genes_unexpressed_TAV.txt")
colnames (genes_Unexpressed_TAV) <- c("gene","transcipt_ID","FPKM_TAV_Rep_1","FPKM_TAV_Rep_2")
genes_Unexpressed_TAV$gene <- as.character(genes_Unexpressed_TAV$gene)


genes_Unexpressed_SMC <- read.delim("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/expression/genes_unexpressed_SMC.txt")
colnames (genes_Unexpressed_SMC) <- c("gene","transcript_ID","FPKM_SMC_Rep_1","FPKM_SMC_Rep_2")
genes_Unexpressed_SMC$gene <- as.character(genes_Unexpressed_SMC$gene)


Comm_Unexpressed_genes = Reduce(intersect,  list(genes_Unexpressed_THP1$gene, 
                                               genes_Unexpressed_TAV$gene,
                                               genes_Unexpressed_SMC$gene))

write.table(unique(Comm_Unexpressed_genes),"Common_UnExpressed_gene.txt",sep="\t", quote=FALSE,
            row.names=FALSE, col.names = FALSE)

Dubious_genes_expression = intersect(Comm_expressed_genes,Comm_Unexpressed_genes ) 
#####################################################################################################################################################
setwd("/Volumes/Work_drive/prj/Expression_HiCap/data/raw_external/")

Common_exprss_and_probe <- intersect(Comm_probe_genes,Comm_expressed_genes)
Common_Unexprss_and_probe <- intersect(Comm_probe_genes,Comm_Unexpressed_genes)


write.table(unique(Common_exprss_and_probe),"Common_Expr_probe_gene.txt",sep="\t", quote=FALSE,
            row.names=FALSE, col.names = FALSE)
write.table(unique(Comm_Unexpressed_genes),"Common_UnExpr_probe_gene.txt",sep="\t", quote=FALSE,
            row.names=FALSE, col.names = FALSE)
#####################################################################################################################################################
######################################################################################################################################################

tmp <- Genes_target_THP1[which(Genes_target_THP1$V1 %in% Common_exprss_and_probe),]
SP_FPKM_THP1 <- tmp[order(tmp[,1]),][,c(1,2,3)]

tmp<- Genes_target_TAV[which(Genes_target_TAV$V1 %in% Common_exprss_and_probe),]
SP_FPKM_TAV <- tmp[order(tmp[,1]),][,c(1,2,3)]

tmp <- Genes_target_SMC[which(Genes_target_SMC$V1 %in% Common_exprss_and_probe),]
SP_FPKM_SMC <- tmp[order(tmp[,1]),][,c(1,2,3)]

head (SP_FPKM_SMC)
tail (SP_FPKM_SMC)

head (SP_FPKM_TAV)
tail (SP_FPKM_TAV)

head (SP_FPKM_THP1)
tail (SP_FPKM_THP1)

All_interaction_combine <- cbind(SP_FPKM_THP1,SP_FPKM_TAV[,c(2,3)],SP_FPKM_SMC[,c(2,3)])
colnames (All_interaction_combine) <- c("Genes","SP_RPKM_Rep1_THP1","SP_RPKM_Rep2_THP1",
                                        "SP_RPKM_Rep1_HAEC","SP_RPKM_Rep2_HAEC",
                                        "SP_RPKM_Rep1_HSMC","SP_RPKM_Rep2_HSMC")



row_sub = apply(All_interaction_combine[2:7], 1, function(row) all(row !=0 ))
All_interaction_combine_clean <- All_interaction_combine[row_sub,]

library(factoextra)
log_TF <- log10(All_interaction_combine_clean[,2:7])
head (log_TF)
tr <- t(log_TF)
colnames(tr) <- All_interaction_combine_clean[, 1]

ir.pca <- prcomp(tr  ,
                 center = TRUE,
                 scale. = TRUE)
ir.genes <- row.names(tr)
cal= c("mTHP1_Rep","mTHP1_Rep",
       "HAEC_Rep", "HAEC_Rep",
       "HSMC_Rep","HSMC_Rep")

fviz_eig(ir.pca)
print(ir.pca)
plot(ir.pca, type = "l")
summary(ir.pca)
plot(ir.pca$x[,1:2],xlab = "PC1", ylab = "PC2", col =as.factor(cal), pch =19)
biplot(ir.pca)


fviz_pca_biplot(ir.pca,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969", # Individuals color
                repel = TRUE     # Avoid text overlapping
)

lab= c("mTHP1","mTHP1","HAEC","HAEC","HSMC","HSMC")

p = fviz_pca_ind(ir.pca, pointsize = 5, habillage=lab, addEllipses=TRUE, ellipse.level=0.95)+
  labs(title ="PCA")+
  xlim(-25, 25) + ylim (-15, 15)

p+ scale_color_brewer(palette="Dark2") +
  theme(legend.text=element_text(size=20),
        legend.position = 'right',
        legend.key.size = unit(1.5, 'lines'),
        axis.title.x = element_text(size=30,face="bold", margin = margin(t = 20, r = 0, b =0, l = 0)),
        axis.text.x = element_text(size=20),
        axis.title.y = element_text(size=30,face="bold",margin = margin(t = 0, r = 20, b =0, l = 0)),
        axis.text.y = element_text(size=20),
        plot.title = element_text(colour="grey20",size=30,hjust=0.5),
        plot.margin = unit(c(2, 2, 2, 2), "cm"))+
  ggtitle("\nPCA plot of significant interaction in all three cell types and replicates \n")
